### This notebook describes the computations to reproduce the correlations from tables 1 and 2.

There are 6 steps:

(Run the following 2 steps outside of this notebook)

1. Download and format the datasets.
- Run the shell script *feature_extraction.sh* from your terminal.

(The following 5 steps are implemented in this notebook)

3. Load features as extracted with *feature_extraction.sh*
- Training an SVM on the features extracted from the LIVE database. We repeat this 10 times per codebook model, every time using a different random train/test split.
- Cross-database evaluation: We load the trained SVMs and evaluate them on the features extracted from TID2013 and CSIQ.
- Computation of correlations: Correlations are computed per train/test split and then averaged across splits. We compute correlations on the full datasets as well as on distortion specific subsets.
- Print results tables (Tables 1 and 2 in the paper)

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler 
from sklearn.feature_selection import SelectFromModel
from scipy.stats import pearsonr, spearmanr
from joblib import dump, load
from tqdm.auto import tqdm

In [ ]:
# Names of reference images in validation set for training/validating.
# Every row corresponds to one split.
val_names = [['buildings', 'statue', 'woman', 'monarch', 'paintedhouse','lighthouse2'],
             ['ocean', 'sailing3', 'caps', 'lighthouse', 'bikes', 'studentsculpture'],
             ['monarch', 'studentsculpture', 'parrots', 'stream', 'sailing3', 'sailing1'],
             ['coinsinfountain', 'manfishing', 'rapids', 'cemetry', 'building2', 'monarch'],
             ['parrots', 'buildings', 'woman', 'dancers', 'sailing3', 'carnivaldolls'],
             ['lighthouse2', 'building2', 'stream', 'ocean', 'woman', 'rapids'],
             ['sailing2', 'lighthouse2', 'parrots', 'manfishing', 'dancers', 'stream'],
             ['buildings', 'coinsinfountain', 'manfishing', 'sailing2','dancers', 'monarch'],
             ['plane', 'monarch', 'sailing3', 'carnivaldolls', 'lighthouse', 'womanhat'],
             ['coinsinfountain', 'caps', 'monarch', 'house', 'ocean', 'churchandcapitol']]

#### 3. Loading extracted features

In [ ]:
# load data (requires steps 1 and 2 to be completed)
data = pd.read_pickle("./features.pkl")
data = data.loc[:,~data.columns.str.contains('Unnamed')] 

#### 4. Training on LIVE

In [ ]:
# All predictions will be save in results.
# This makes it easy to evaluate correlations on different subsets later on.
results = pd.DataFrame()

for tqdm(split in range(10)): # random splits
    for model in tqdm(sorted(data.codebook.unique()), leave=False): # codebook models
        
        # Create dir to save trained svr model in
        log_dir = "./regression_models"
        if not os.path.exists(log_dir):
            os.makedirs(log_dir)

        # Select data
        idcs = (data.codebook == model) & \
               (data.dataset == "liveiqa")

        # Split data (there are predefined splits)
        data_train = pd.DataFrame(columns=data.columns.tolist() + ["modus", "split", "preds"])
        data_train = data_train.append(data.loc[idcs & (~data.refname.isin(val_names[split]))])
                
        data_val = pd.DataFrame(columns=data.columns.tolist() + ["modus", "split", "preds"])
        data_val = data_val.append(data.loc[idcs & (data.refname.isin(val_names[split]))])

        # Get features
        betas_train = np.vstack(data_train.beta.values)
        betas_val = np.vstack(data_val.beta.values)

        # On training data, find parameters to scale features to the range [-1, 1]
        scaler = MinMaxScaler(feature_range=[-1,1])
        scaler.fit(betas_train)
        dump(scaler, os.path.join(log_dir, "minmaxscaler_{}_{}.joblib".format(model, split)))
        
        # Apply parameters to train and test data
        betas_train = scaler.transform(betas_train)
        betas_val = scaler.transform(betas_val)

        # Fit and save support vector machine
        svr = svm.NuSVR(kernel='linear', C=1.0, nu=0.5, cache_size=1000)
        svr.fit(betas_train, data_train.q_norm)
        dump(svr, os.path.join(log_dir, "svr_{}_{}.joblib".format(model, split)))
        
        # Save results on training set
        data_train.loc[:, "modus"] = "train"
        data_train.loc[:, "split"] = split
        data_train.loc[:, "preds"] = svr.predict(betas_train)

        # Save results on test set
        data_val.loc[:, "modus"] = "test"
        data_val.loc[:, "split"] = split
        data_val.loc[:, "preds"] = svr.predict(betas_val)            

        # Save results in dataFrame
        results = results.append(data_train, ignore_index=True)
        results = results.append(data_val, ignore_index=True)
        
results.to_pickle("./predictions.pkl")

#### 5. Cross-database evaluation on TID2013 and CSIQ

In [ ]:
tid_names_unique = ["i01", "i02", "i07", "i12", "i15", "i25"]

data.loc[data.refname == "i03", "refname"] = "caps"
data.loc[data.refname == "i04", "refname"] = "womanhat"
data.loc[data.refname == "i05", "refname"] = "bikes"
data.loc[data.refname == "i06", "refname"] = "sailing1"
data.loc[data.refname == "i08", "refname"] = "buildings"

data.loc[data.refname == "i09", "refname"] = "sailing2"
data.loc[data.refname == "i10", "refname"] = "sailing3"
data.loc[data.refname == "i11", "refname"] = "sailing4"
data.loc[data.refname == "i13", "refname"] = "stream"
data.loc[data.refname == "i14", "refname"] = "rapids"

data.loc[data.refname == "i16", "refname"] = "ocean"
data.loc[data.refname == "i17", "refname"] = "statue"
data.loc[data.refname == "i18", "refname"] = "woman"
data.loc[data.refname == "i19", "refname"] = "lighthouse"
data.loc[data.refname == "i20", "refname"] = "plane"

data.loc[data.refname == "i21", "refname"] = "lighthouse2"
data.loc[data.refname == "i22", "refname"] = "house"
data.loc[data.refname == "i23", "refname"] = "parrots"
data.loc[data.refname == "i24", "refname"] = "paintedhouse"

data.loc[data.distortion == "wn", "distortion"] = "awgn"

In [ ]:
for split in tqdm(range(10)): # random splits
    for model in tqdm(sorted(data.codebook.unique()), leave=False): # codebook models
        
        # Select data
        idcs = (data.codebook == model) & \
               (data.dataset == "tid2013")
        
        # Create dataFrame for tid
        data_tid = pd.DataFrame(columns=data.columns.tolist() + ["modus", "split", "preds"])
        # Avoid content spill - only use reference images not contained in training set
        data_tid = data_tid.append(data.loc[idcs & (data.refname.isin(val_names[split] + tid_names_unique))])
        
        # Select data
        idcs = (data.codebook == model) & \
               (data.dataset == "csiq")
        
        # Create dataFrame for csiq
        data_csiq = pd.DataFrame(columns=data.columns.tolist() + ["modus", "split", "preds"])
        # We can use all image as LIVE and CSIQ do not share any reference images
        data_csiq = data_csiq.append(data.loc[idcs])

        # Get features
        betas_tid = np.vstack(data_tid.beta.values)
        betas_csiq = np.vstack(data_csiq.beta.values)
        
        scaler = load(os.path.join(log_dir, "minmaxscaler_{}_{}.joblib".format(model, split)))
        
        # Apply parameters to test data
        betas_tid = scaler.transform(betas_tid)
        betas_csiq = scaler.transform(betas_csiq)

        svr = load(os.path.join(log_dir, "svr_{}_{}.joblib".format(model, split)))
        
        # Save results on tid test set
        data_tid.loc[:, "modus"] = "test"
        data_tid.loc[:, "split"] = split
        data_tid.loc[:, "preds"] = svr.predict(betas_tid)            
        
        # Save results on csiq test set
        data_csiq.loc[:, "modus"] = "test"
        data_csiq.loc[:, "split"] = split
        data_csiq.loc[:, "preds"] = svr.predict(betas_csiq)            

        # Save results in dataFrame
        results = results.append(data_tid, ignore_index=True)
        results = results.append(data_csiq, ignore_index=True)
        
        # Compute correlation - this is only a early on sanity check to see if everything is working
        # Actual evaluation is done below
        pcc_tid = pearsonr(data_tid.loc[:, "preds"], data_tid.q_norm)[0]
        pcc_csiq = pearsonr(data_csiq.loc[:, "preds"], data_csiq.q_norm)[0]
        
results.to_pickle("./predictions.pkl")

#### 6. Computation of correlations

In [ ]:
results.loc[results.distortion == "wn", "distortion"] = "awgn"

# Setting up the correlation tables
corr_columns = ["pc_full", "sc_full",
                "pc_jpeg", "sc_jpeg",
                "pc_jp2k", "sc_jp2k",
                "pc_gblur", "sc_gblur",
                "pc_awgn", "sc_awgn",
                "pc_shared", "sc_shared"]

correlations = pd.DataFrame(columns=["model", "dataset"] + corr_columns)

# Distortion types considered in the paper
dists = ["full", "jpeg", "jp2k", "gblur", "awgn", "shared"]

for db in tqdm(results.dataset.unique()):
    for codebook in tqdm(["cornia", "patches", "laplace", "normal", "uniform"], leave=False):
        for dist in tqdm(dists, leave=False):
            pccs, sroccs = [], []
            for split in results.split.unique():

                if dist == "full":
                    _dists = results.loc[results.dataset == db].distortion.unique()
                elif dist == "shared":
                    _dists = ["jpeg", "jp2k", "gblur", "awgn"]
                else:
                    _dists = [dist]
                
                
                # Select predictions of this split
                idcs = (results.codebook == codebook) & \
                       (results.dataset == db) & \
                       (results.split == split) & \
                       (results.modus == "test") & \
                       (results.distortion.isin(_dists))

                if not np.any(idcs): 
                    continue

                # Compute correlations between quality predictions and quality annotations
                pccs.append(pearsonr(results.loc[idcs].preds, results.loc[idcs].q_norm)[0])
                sroccs.append(spearmanr(results.loc[idcs].preds, results.loc[idcs].q_norm)[0])
            
            # Save correlations
            row_idx = (correlations.dataset == db) & (correlations.model == codebook)
                      
            if not np.any(row_idx):
                row_idx = correlations.shape[0]
                
            correlations.loc[row_idx, "dataset"] = db
            correlations.loc[row_idx, "model"] = codebook
            correlations.loc[row_idx, "pc_{}".format(dist)] = np.mean(pccs)
            correlations.loc[row_idx, "sc_{}".format(dist)] = np.mean(sroccs)
            
correlations[corr_columns] = correlations[corr_columns].apply(pd.to_numeric)
correlations.to_pickle("correlations.pkl")

#### 7. Print results tables (Tables 1 and 2 in the paper)

In [ ]:
print(correlations.loc[correlations.dataset == "liveiqa"].round(decimals=2))

In [ ]:
print(correlations.loc[correlations.dataset == "tid2013"].round(decimals=2))

In [ ]:
print(correlations.loc[correlations.dataset == "csiq"].round(decimals=2))